In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

In [2]:
from scipy.constants import m_p, c, e

In [3]:
from cpymad.madx import Madx

import sixtracklib as pyst
import pysixtrack

# I. MAD-X part

In [4]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False
madx.options.info = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.10.25 11:39:36      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [56]:
p0c = 1e9 # in eV
Etot = np.sqrt(p0c**2 + (m_p/e)**2 * c**4) * 1e-9 # in GeV

Define a simple sequence in MAD-X with quadrupolar and sextupolar component:

In [57]:
madx.input('''
qd: multipole, knl := {0, 0.1, 2};

line: sequence, l = 1;
qd, at = 0.5;
endsequence;
''')

madx.command.beam(particle='proton', energy=Etot) # energy in GeV

madx.use(sequence='line')

## I.1. track through `line` without errors:

tracking:

In [58]:
madx.command.track(onetable=True, onepass=True)
madx.command.start(x=0.01, y=0.005)
madx.command.run()
madx.command.endtrack()

enter TRACK module
one pass is on
exit TRACK module



True

relative change of $x$ coordinate:

In [59]:
x_noerr = madx.table.trackone[1]['x'] / madx.table.trackone[0]['x']
x_noerr

0.9462499592675317

## I.2. now track with tilt and offset error:

In [60]:
dpsi = np.pi/6
dx = dy = 0.01

Apply this `dpsi` tilt error to the quadrupole:

In [61]:
madx.select(flag='error', pattern='qd')
madx.command.ealign(dpsi=dpsi, dx=dx, dy=dy)

qd = madx.sequence.line.expanded_elements[2]

qd.align_errors.dpsi, qd.align_errors.dx, qd.align_errors.dy

(0.5235987755982988, 0.01, 0.01)

tracking:

In [62]:
madx.command.track(onetable=True, onepass=True)
madx.command.start(x=0.01, y=0.005)
madx.command.run()
madx.command.endtrack()

enter TRACK module
one pass is on
exit TRACK module



True

relative change of $x$ coordinate:

In [63]:
x_err = madx.table.trackone[1]['x'] / madx.table.trackone[0]['x']
x_err

1.02165063767239

In comparison to no error:

In [64]:
x_err / x_noerr

1.0796836794193618

$\implies$ significant effect of `dpsi` tilt and `dx` / `dy` offset error on x coordinate

# II. SixTrackLib part

## II.1. without tilt and offset error:

In [65]:
elements = pyst.Elements.from_mad(madx.sequence.line, exact_drift=True)
elements.BeamMonitor(num_stores=1);

In [66]:
particles = pyst.Particles.from_ref(1, p0c=p0c)

In [67]:
particles.x = 0.01
particles.y = 0.005

In [68]:
job = pyst.TrackJob(elements, particles)

In [69]:
job.track(1)

job.collect()

job.output.particles[0]

<Particles at 128
  num_particles:1
  q0:[1.]
  mass0:[9.38272081e+08]
  beta0:[0.72925621]
  gamma0:[1.46147393]
  p0c:[1.e+09]
  s:[1.]
  x:[0.0094625]
  y:[0.0053]
  px:[-0.001075]
  py:[0.0006]
  zeta:[-3.78906681e-07]
  psigma:[0.]
  delta:[0.]
  rpp:[1.]
  rvv:[1.]
  chi:[1.]
  charge_ratio:[1.]
  particle_id:[0]
  at_element:[5]
  at_turn:[0]
  state:[1]
>

In [70]:
particles.x[0] / 0.01

0.9462499592675317

Comparing to `x_noerr` from the MAD-X tracking above:

In [71]:
x_noerr == particles.x[0] / 0.01

True

$\implies$ `SixTrackLib` and `MAD-X` agree exactly on the tracking for the single particle!

## II.2. with tilt and offset error:

Create Line in `PySixTrack` in order to manipulate it. Then add the `SRotation` element for the tilt and `XYShift` element for the offset:

In [87]:
pysixtrack_elements, _ = pysixtrack.line.Line.from_madx_sequence(
    madx.sequence.line, exact_drift=True)

In [88]:
pysixtrack_elements.elements

[DriftExact(length=0.0),
 DriftExact(length=0.5),
 Multipole(knl=[0.0, 0.1, 2.0], ksl=[0.0], hxl=0.0, hyl=0, length=0.0),
 DriftExact(length=0.5),
 DriftExact(length=0.0)]

Define the tilt in `PySixTrack`:

In [89]:
angle = dpsi * 180 / np.pi

srot = pysixtrack.elements.SRotation(angle=angle)
inv_srot = pysixtrack.elements.SRotation(angle=-angle)

In [90]:
pysixtrack_elements.elements.insert(2, srot)
pysixtrack_elements.elements.insert(4, inv_srot)
pysixtrack_elements.elements

[DriftExact(length=0.0),
 DriftExact(length=0.5),
 SRotation(angle=29.999999999999996),
 Multipole(knl=[0.0, 0.1, 2.0], ksl=[0.0], hxl=0.0, hyl=0, length=0.0),
 SRotation(angle=-29.999999999999996),
 DriftExact(length=0.5),
 DriftExact(length=0.0)]

Define the offset in `PySixTrack`:

In [91]:
xyshift = pysixtrack.elements.XYShift(dx=dx, dy=dy)
inv_xyshift = pysixtrack.elements.XYShift(dx=-dx, dy=-dy)

In [92]:
pysixtrack_elements.elements.insert(2, xyshift)
pysixtrack_elements.elements.insert(6, inv_xyshift)
pysixtrack_elements.elements

[DriftExact(length=0.0),
 DriftExact(length=0.5),
 XYShift(dx=0.01, dy=0.01),
 SRotation(angle=29.999999999999996),
 Multipole(knl=[0.0, 0.1, 2.0], ksl=[0.0], hxl=0.0, hyl=0, length=0.0),
 SRotation(angle=-29.999999999999996),
 XYShift(dx=-0.01, dy=-0.01),
 DriftExact(length=0.5),
 DriftExact(length=0.0)]

Pass over to `SixTrackLib`:

In [93]:
elements = pyst.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=1);

In [94]:
particles = pyst.Particles.from_ref(1, p0c=p0c)

In [95]:
particles.x = 0.01
particles.y = 0.005

In [96]:
job = pyst.TrackJob(elements, particles)

In [97]:
job.track(1)

job.collect()

job.output.particles[0]

<Particles at 128
  num_particles:1
  q0:[1.]
  mass0:[9.38272081e+08]
  beta0:[0.72925621]
  gamma0:[1.46147393]
  p0c:[1.e+09]
  s:[1.]
  x:[0.01021651]
  y:[0.0048875]
  px:[0.00043301]
  py:[-0.000225]
  zeta:[-5.95312606e-08]
  psigma:[0.]
  delta:[0.]
  rpp:[1.]
  rvv:[1.]
  chi:[1.]
  charge_ratio:[1.]
  particle_id:[0]
  at_element:[9]
  at_turn:[0]
  state:[1]
>

In [98]:
particles.x[0] / 0.01

1.0216506376723902

Comparing to `x_err` from the MAD-X tracking above, the relative error gives:

In [102]:
(x_err - particles.x[0] / 0.01) / x_err

-2.1733907535252162e-16

$\implies$ offset and tilt error at the same time work!